In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from tensorflow.keras.layers import Dense, LSTM, Dropout, Activation, BatchNormalization, LayerNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")

In [ ]:
df.drop("date_time",axis=1,inplace=True)


In [ ]:
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
ind = test["date_time"]
test.drop("date_time",axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
for i in df.columns:
    sns.histplot(df[i])
    plt.show()

In [ ]:
df1 = np.log(df.iloc[:,:-3].copy()+100)
test = np.log(test+100)
for i in df1.columns[:-3]:
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    sns.histplot(x=i,data=df1,kde=True,ax=ax[0])  
    sns.histplot(x=i,data=df,color="green",kde=True,ax=ax[1])
    # plt.legend()
    plt.show()

In [ ]:
for i in df1.columns[:-3]:
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    sns.boxplot(x=i,data=df1,ax=ax[0])  
    sns.boxplot(x=i,data=df,color="green",ax=ax[1])
    # plt.legend()
    plt.show()

In [ ]:
from scipy.stats import probplot
for i in df1.columns[:-3]:
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    probplot(df1[i],dist='norm',plot=ax[0],)  
    probplot(df[i],dist='norm',plot=ax[1])
    # plt.legend()
    plt.show()

In [ ]:

def remove_outlier_IQR(df):
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df_final = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_final

In [ ]:
df3 = df1.copy()
df3 = df3.join(df.iloc[:,-3:])
df3 = remove_outlier_IQR(df3)
test3 = test.copy()
# test3 = remove_outlier_IQR(test3)

In [ ]:
df3

In [ ]:
# model = Sequential()                # could make units 50, for better prediction
# # model.add(LayerNormalization())     # Normalizing the input data 
# # First LSTM layer
# model.add(LSTM(units=50,return_sequences=True,input_shape=(8,1),activation="relu",unroll=True))
# # model.add(Dropout(0.1))    # Removing some of the data from the previous layer to avoid overfitting
# model.add(BatchNormalization())
# model.add(LSTM(units=50, activation="relu",unroll=True,return_sequences=True))#, return_sequences=True))    # Second LSTM Layer
# # model.add(Dropout(0.1))
# model.add(Activation(activation="relu"))
# model.add(BatchNormalization())
# model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# model.add(BatchNormalization())
# model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# model.add(BatchNormalization())
# model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# model.add(BatchNormalization())
# model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# # model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# # model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# # model.add(LSTM(units=50, return_sequences=True))
# # model.add(Dropout(0.1))
# model.add(Activation(activation="relu"))
# model.add(BatchNormalization())
# model.add(LSTM(units=50))
# # model.add(Dropout(0.1))
# model.add(BatchNormalization())
# ## 
# # model.add(Dropout(0.1))
# # model.add(LSTM(units=50,activation="relu"))
# # model.add(Dense(units=500, activation="relu"))
# # model.add(Dropout(0.2))

# model.add(Dense(units=1,activation='linear'))  # Final LSTM Layer of unit=1, cause we have just one Output, that is, Closing Price
# model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=1))   # Compiling, optimizer and loss can be changed to optimize the model for our specific purpose
# model.fit(np.array(df3.iloc[:,:-3]).reshape(7111,-1,1),np.array(df3["target_carbon_monoxide"]).reshape(-1,1), epochs=10, batch_size=100)

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [ ]:
test.isna().sum()

In [ ]:
scaler = RobustScaler()
X = scaler.fit_transform(df3.iloc[:,:-3])
test3 = scaler.transform(test3)

In [ ]:
scaler1 = RobustScaler().fit_transform(np.array(df3["target_carbon_monoxide"]).reshape(-1,1))

In [ ]:
x_train,x_test, y_train,y_test = train_test_split(X,df3["target_carbon_monoxide"])

In [ ]:
from xgboost import XGBRFRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(x_train,y_train,verbose=False)

In [ ]:
model.predict(test)

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(model.predict(x_train),y_train,squared=False))
print(mean_squared_error(model.predict(x_test),y_test,squared=False))

In [ ]:
# scaler1 = RobustScaler()
# Y = scaler1.fit_transform(np.array(df3["target_carbon_monoxide"]).reshape(-1,1))
# model.fit(X,Y,verbose=False)
# y_pred1 = []
# y_pred1 = scaler1.inverse_transform(model.predict(test3).reshape(-1,1))


model.fit(X,df3["target_carbon_monoxide"],verbose=False)
y_pred1 = []
y_pred1 = model.predict(test3)

In [ ]:
y_pred1

In [ ]:
# scaler1 = RobustScaler()
# Y = scaler1.fit_transform(np.array(df3["target_benzene"]).reshape(-1,1))
# model.fit(X,Y,verbose=False)
# y_pred2=[]
# y_pred2 =scaler1.inverse_transform(model.predict(test3).reshape(-1,1))

model.fit(X,df3["target_benzene"],verbose=False)
y_pred2 = []
y_pred2 = model.predict(test3)

In [ ]:
# scaler1 = RobustScaler()
# Y = scaler1.fit_transform(np.array(df3["target_nitrogen_oxides"]).reshape(-1,1))
# model.fit(X,Y,verbose=False)
# y_pred3=[]
# y_pred3 =scaler1.inverse_transform(model.predict(test3).reshape(-1,1))

model.fit(X,df3["target_nitrogen_oxides"],verbose=False)
y_pred3 = []
y_pred3 = model.predict(test3)

In [ ]:
final = pd.DataFrame({"date_time":np.array(ind).reshape(-1), "target_nitrogen_oxides":y_pred3.reshape(-1), "target_benzene":y_pred2.reshape(-1), "target_carbon_monoxide":y_pred1.reshape(-1)})

In [ ]:
final.to_csv("submission.csv")

In [ ]:
final.set_index("date_time",inplace=True)

In [ ]:
final.to_csv("submission1.csv")